# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

/Users/jingle.jiang/anaconda2/envs/cs231n/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
from davis import *

## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [3]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9

## Create Model and Load Trained Weights

In [4]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [5]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Run Object Detection

In [6]:
# Load a random image from the images folder
# file_names = next(os.walk(IMAGE_DIR))[2]
# image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
file_names = "00000.jpg"
IMAGE_DIR = "/Users/jingle.jiang/personal/class/stanford/\
cs231n/final/video_segmentation/davis-2017/data/DAVIS/JPEGImages/480p/elephant"

image = skimage.io.imread(os.path.join(IMAGE_DIR, file_names))

In [ ]:
r['rois'][:1].shape[0]

In [ ]:
r['masks'][:,:,:1].shape

In [ ]:
visualize.display_instances(image, r['rois'][:1], r['masks'][:,:,:1], r['class_ids'][:1], 
                            class_names, r['scores'][:1])

In [ ]:


# Run detection
# results = model.detect([image], verbose=1)

# Visualize results
r = result_
visualize.display_instances(image, r['rois'][1:], r['masks'][:,:,1:], r['class_ids'][1:], 
                            class_names, r['scores'][1:])

In [ ]:
data_ = np.squeeze(data[:, :, 0])
for i in range(1, data.shape[2]):
    data_[np.squeeze(data[:, :, i]) != 0] = (i+1)
data = data_

In [ ]:
io.imwrite_indexed(os.path.join(SAVE_DIR, file_[:-3]+"png"), data)

In [23]:
FOLDER_NAMES = [_ for _ in cfg.SEQUENCES.keys()]

In [24]:
FOLDER_NAMES

['aerobatics',
 'bear',
 'bike-packing',
 'blackswan',
 'bmx-bumps',
 'bmx-trees',
 'boat',
 'boxing-fisheye',
 'breakdance',
 'breakdance-flare',
 'bus',
 'camel',
 'car-race',
 'car-roundabout',
 'car-shadow',
 'car-turn',
 'carousel',
 'cat-girl',
 'cats-car',
 'chamaleon',
 'classic-car',
 'color-run',
 'cows',
 'crossing',
 'dance-jump',
 'dance-twirl',
 'dancing',
 'deer',
 'disc-jockey',
 'dog',
 'dog-agility',
 'dog-gooses',
 'dogs-jump',
 'dogs-scale',
 'drift-chicane',
 'drift-straight',
 'drift-turn',
 'drone',
 'elephant',
 'flamingo',
 'giant-slalom',
 'girl-dog',
 'goat',
 'gold-fish',
 'golf',
 'guitar-violin',
 'gym',
 'helicopter',
 'hike',
 'hockey',
 'horsejump-high',
 'horsejump-low',
 'horsejump-stick',
 'hoverboard',
 'india',
 'judo',
 'kid-football',
 'kite-surf',
 'kite-walk',
 'koala',
 'lab-coat',
 'lady-running',
 'libby',
 'lindy-hop',
 'loading',
 'lock',
 'longboard',
 'lucia',
 'mallard-fly',
 'mallard-water',
 'man-bike',
 'mbike-trick',
 'miami-surf',


In [7]:
# let's do this iteratively
SAVE_DIR = "/Users/jingle.jiang/personal/class/\
stanford/cs231n/final/video_segmentation/davis-2017/data/DAVIS/MaskRCNN/480p/elephant/"
images = []
files = []
count = -1
for file in os.listdir(IMAGE_DIR):
    count += 1
    if count <= 5:
        continue
    if file.endswith(".jpg"):
        print(file + " out of " + str(len(os.listdir(IMAGE_DIR))))
        image = skimage.io.imread(os.path.join(IMAGE_DIR, file))
        images.append(image)
        files.append(file)
        if len(images) == config.IMAGES_PER_GPU:
            results = model.detect(images, verbose=0)
            #save
            for file_, result_ in zip(files, results):
                data = np.squeeze(result_["masks"])
                data = data.astype('uint8')
                if np.atleast_3d(data).shape[2] != 1:
                    #when we detect more then one items
                    data_ = np.squeeze(data[:, :, 0])
                    for i in range(1, data.shape[2]):
                        data_[np.squeeze(data[:, :, i]) != 0] = (i+1)
                    data = data_
                io.imwrite_indexed(os.path.join(SAVE_DIR, file_[:-3]+"png"), data)
            
            # before done, reinit images and files
            images = []
            files = []


00006.jpg out of 80
00007.jpg out of 80
00008.jpg out of 80
00009.jpg out of 80
00010.jpg out of 80
00011.jpg out of 80
00012.jpg out of 80
00013.jpg out of 80
00014.jpg out of 80
00015.jpg out of 80
00016.jpg out of 80
00017.jpg out of 80
00018.jpg out of 80
00019.jpg out of 80
00020.jpg out of 80
00021.jpg out of 80
00022.jpg out of 80
00023.jpg out of 80
00024.jpg out of 80
00025.jpg out of 80
00026.jpg out of 80
00027.jpg out of 80
00028.jpg out of 80
00029.jpg out of 80
00030.jpg out of 80
00031.jpg out of 80
00032.jpg out of 80
00033.jpg out of 80
00034.jpg out of 80
00035.jpg out of 80
00036.jpg out of 80
00037.jpg out of 80
00038.jpg out of 80
00039.jpg out of 80
00040.jpg out of 80
00041.jpg out of 80
00042.jpg out of 80
00043.jpg out of 80
00044.jpg out of 80
00045.jpg out of 80
00046.jpg out of 80
00047.jpg out of 80
00048.jpg out of 80
00049.jpg out of 80
00050.jpg out of 80
00051.jpg out of 80
00052.jpg out of 80
00053.jpg out of 80
00054.jpg out of 80
00055.jpg out of 80


In [9]:
len(os.listdir(IMAGE_DIR))

80

In [ ]:
array = data
np.atleast_3d(array).shape[2] != 1
np.unique(array)

In [ ]:
io.imwrite_indexed(os.path.join(SAVE_DIR, file_[:-3]+"png"), data)

In [ ]:
np.unique(results[0]["masks"])

In [ ]:
data = np.squeeze(results[0]["masks"])
data = data.astype('uint8')
np.unique(data)

In [ ]:
file_names = 'elephant/00000.png'

In [ ]:
SAVE_DIR = "/Users/jingle.jiang/personal/class/stanford/cs231n/final/video_segmentation/davis-2017/data/DAVIS/MaskRCNN/480p/"
image = io.imwrite_indexed(os.path.join(SAVE_DIR, file_names), data)

In [ ]:
os.path.join(SAVE_DIR, file_names)

In [ ]:
io

In [ ]:
for file in range(80):
    if file<10:
        file_names = 'elephant/0000' + str(file) + ".png"
    else:
        file_names = 'elephant/000' + str(file) + ".png"
    image = io.imwrite_indexed(os.path.join(SAVE_DIR, file_names), data)

In [ ]:
ANN_DIR = "/Users/jingle.jiang/personal/class/stanford/cs231n/final/video_segmentation/davis-2017/data/DAVIS/Annotations/480p/"
ann = skimage.io.imread(os.path.join(ANN_DIR, "elephant/00000.png"))

In [ ]:
ann.shape

In [ ]:
np.unique(ann)

In [ ]:
ANN_DIR = "/Users/jingle.jiang/personal/class/stanford/cs231n/final/video_segmentation/davis-2017/data/DAVIS/MaskRCNN/480p/"
ann = skimage.io.imread(os.path.join(ANN_DIR, "elephant/00000.png"))
np.unique(ann)

In [ ]:
ann.shape

In [ ]:
ann[200:300, 300:400]